# PTT 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 印出最新文章的「作者」「標題」「時間」
* ② 印出第一頁所有文章的「作者」「標題」「時間」


In [1]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/NBA/index.html'
r = requests.get(url)

soup = BeautifulSoup(r.text, "html5lib")

### ① 印出最新文章的「作者」「標題」「時間」

In [3]:
for d in soup.find_all(class_="title"):
    print(d.text.replace('\t', '').replace('\n', ''))
    try:
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        print('作者: ' + r.find(class_='article-meta-value').text)
        print('時間: ' + r.find_all(class_='article-meta-value')[2].text)
    except:
        continue


[情報] Reggie Miller對下季湖人的看法
作者: jackie0414 (哇哈哈)
時間: [外絮] Reggie Miller對下季湖人的看法
[討論] Rubio在NBA懷才不遇嗎
作者: ChrisBosh (Bosh)
時間: [討論] Rubio在NBA懷才不遇嗎
[情報] Kelly Oubre Jr.跟黃蜂達成協議
作者: arod1414 (Kawhi Leonard頭號粉絲)
時間: [情報] Kelly Oubre Jr.跟黃蜂達成協議
[情報] DeMar DeRozan加盟公牛的幕後細節
作者: Petyr (小指頭)
時間: [情報] DeMar DeRozan加盟公牛的幕後細節
[新聞] 卡盧索最後一刻不放棄續留 湖人薪資考量
作者: humbler (獸人H)
時間: [新聞] 卡盧索最後一刻不放棄續留 湖人薪資考量
Re: [花邊] 老巴：字母就跟喬丹一樣經歷失敗奪冠
作者: gaowolf (I've been waiting long.)
時間: Re: [花邊] 老巴：字母就跟喬丹一樣經歷失敗奪冠
[公告] 水桶*9 警告*1 不罰*4
作者: pneumo (超☆冒險蓋)
時間: [公告] 水桶*9 警告*1 不罰*4
[花邊] 小李第四張專輯發佈
作者: AlanLinKL (AlanKL)
時間: [花邊] 小李第四張專輯發佈
(已被p刪除) <hualiens> LYS文禁發
[公告] 七月份工作報告
作者: Induction (愛人愛己)
時間: [公告] 七月份工作報告
[花邊] ESPN計算Curry新合約 秒薪$1.7美元
作者: VeryGoodBoy (很棒的男孩)
時間: [花邊] ESPN計算Curry新合約 秒薪$1.7美元
[新聞] 拒絕冠軍隊公鹿、湖人　魯長老重返老鷹
作者: arod1414 (Kawhi Leonard頭號粉絲)
時間: [新聞] 拒絕冠軍隊公鹿、湖人　魯長老重返老鷹
[討論] 熱火今年有機會衝擊東冠嗎？
作者: ya870323 (忘憂的小夜曲)
時間: [討論] 熱火今年有機會衝擊東冠嗎？
[花邊] 為了美國隊？東奧男籃金牌戰竟在銅牌戰前
作者: u9300017 (載雨拾乘三)
時間: [花邊] 為了美國隊？東奧男籃金牌戰竟在銅牌戰前
Re: [花邊] 老巴：

### ② 印出第一頁所有文章的「作者」「標題」「時間」

In [ ]:
'''
Your Code
'''

### ③ 試著爬爬看其他版的文章

In [ ]:
'''
Your Code
'''